# Subset and regrid CMIP6 data from ESGF

In [ ]:
if "google.colab" in str(get_ipython()):
    print("Running on CoLab")
    !pip install zarr==2.18 cftime intake-esgf rooki
else:
    print("Not running on CoLab")

This notebook showcases how to use `intake_esfg` to search and filter the CMIP6 collection, and how to use `rooki` to subset and regrid the data on the cloud.

In [ ]:
import os

import intake_esgf

# Run this on the DKRZ node in Germany, using the ESGF1 index node at LLNL
os.environ["ROOK_URL"] = "http://rook.dkrz.de/wps"
# data download directory
import os

os.environ["ROOKI_OUTPUT_DIR"] = os.path.join(os.getcwd(), "rookie_output")

intake_esgf.conf.set(
    indices={"anl-dev": False, "ornl-dev": False, "esgf-node.llnl.gov": True}
)

import xarray as xr
from intake_esgf import ESGFCatalog
from rooki import operators as ops

## Retrieve subset of CMIP6 data

The CMIP6 dataset is identified by a dataset-id. Using intake-esgf we can query the ESGF database for the variables and models we are interested in. For this demo we are interested in the tos (sea surface temperature) variable for the historical runs. Also, for sake of simplicity we will only query a subset of the models available.

In [ ]:
cat = ESGFCatalog()
cat.search(
    experiment_id=["historical"],
    variable_id=["tos"],
    table_id=["Omon"],
    project=["CMIP6"],
    grid_label=["gn"],
    source_id=[
        "CESM2-FV2",
        "CESM2-WACCM-FV2",
        "FGOALS-f3-L",
        "MIROC-ES2L",
    ],
)
cat.remove_ensembles()  # we only want to work with the parent datasets
print(cat)

Once the catalog has been queried, we have to do some manipulation in pandas to keep only the dataset_id. This has to be done because the same data has multiple locations online, and these get appended at the end of the dataset_id. Rookie only accepts the dataset_id without the online location, so we get rid of it in the next step.

In [ ]:
cat.df.id[0]

In [ ]:
def keep_ds_id(ds):
    return ds[0].split("|")[0]

These paths are what we are looking for

In [ ]:
collections = cat.df.id.apply(keep_ds_id).to_list()
collections

## Subset and regrid
We define a function that will do the subset and regridding for us for each of the dataset_ids we have. The function will take the dataset_id as input and then use Rookie functions to select 100 years of data for the tos variable in the Pacific Ocean region.

For more information about the operations, you can go to [rook's documentation](https://rook-wps.readthedocs.io/en/latest/processes.html#).

For regridding, refer to this [source code](https://github.com/roocs/rook/blob/main/src/rook/processes/wps_regrid.py)


**Note:** Some dataset requests might fail when querying more than 25 years of data (might be size related, needs more testing). So it would be safer to keep the request below that threshold and implement a loop to retrieve more data if needed.

In [ ]:
def get_pacific_ocean(dataset_id):
    wf = ops.Regrid(
        ops.Subset(
            ops.Input("tos", [dataset_id]),
            time="1850-01-01/1875-01-31",
            area="100,-20,280,20",
        ),
        method="bilinear",
        grid="1deg",
    )
    resp = wf.orchestrate()
    if resp.ok:
        print(f"{resp.size_in_mb=}")
        ds = resp.datasets()[0]
    else:
        raise ValueError(resp)
        # ds = xr.Dataset()
    return ds

In [ ]:
sst_data = {dset: get_pacific_ocean(dset) for dset in collections}

The result will be downloaded to a temp folder in our local machine. We can then explore the data using xarray or any other tool of our choice.

In [ ]:
sst_data

## Requesting data with vertical levels

This process should be similar to what we have already explored before. The general steps for requesting data are as follows:

1. Use `intake_esgf` to search for the dataset of interest.
2. Filter the results to get the dataset_ids we want.
3. Use `rooki` to subset and regrid the data.
4. Download the data.

In [ ]:
cat = ESGFCatalog()
cat.search(
    experiment_id=["historical"],
    variable_id=["thetao"],
    table_id=["Omon"],
    project=["CMIP6"],
    grid_label=["gn"],
    source_id=[
        "CESM2-FV2",
        "CESM2-WACCM-FV2",
        "FGOALS-f3-L",
        "MIROC-ES2L",
    ],
)
cat.remove_ensembles()
print(cat)

In [ ]:
cat.df

In [ ]:
collections = cat.df.id.apply(keep_ds_id).to_list()
collections

In [ ]:
def get_pacific_ocean(dataset_id):
    wf = ops.Regrid(
        ops.Subset(
            ops.Input("thetao", [dataset_id]),
            time="1850-01-01/1851-01-31",
            area="100,-10,280,10",
            level="0/50",
        ),
        method="billinear",
        grid="2pt5deg",
    )
    resp = wf.orchestrate()
    if resp.ok:
        print(f"{resp.size_in_mb=}")
        ds = resp.datasets()[0]
    else:
        ds = xr.Dataset()
    return ds

This might take some time to complete depending on the region and time selection. Another convenient method to load this data is to use the google cloud storage bucket, which can be find [here](https://github.com/ckaramp-research/code-snippets/tree/main)

In [ ]:
thetao_data = {dset: get_pacific_ocean(dset) for dset in collections}